# Neural Networks

In [2]:
import time
import os
import pprint
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
import utils

# Network Architecture Definition (nnet1)

In [ ]:
class NNet1(nn.Module):
    def __init__(self):
        super(NNet1, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 128, kernel_size=(4, 513))
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv2 = nn.Conv2d(128, 128, kernel_size=(4, 1))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 1))
        self.conv3 = nn.Conv2d(128, 256, kernel_size=(4, 1))
        self.avgpool = nn.AvgPool2d(kernel_size=(26, 1))
        self.maxpool = nn.MaxPool2d(kernel_size=(26, 1))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(512, 300)
        self.dense2 = nn.Linear(300, 150)
        self.dense3 = nn.Linear(150, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x_avg = self.avgpool(x)
        x_max = self.maxpool(x)
        x = torch.cat([x_avg, x_max], dim=1)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.softmax(x)
        return x

# Dataset Class

In [ ]:
batch_size=16
# Define the custom dataset class
class MyDataset(Dataset):
    def __init__(self, file_list, labels):
        self.file_list = file_list
        self.labels=labels

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_paths = self.file_list[idx:idx+batch_size]  # Get batch of file paths
        labels = self.labels[idx:idx+batch_size]  # Get batch of labels

        stft_vectors = []
        for file_path in file_paths:
            stft_vector = np.load(file_path).transpose(1, 0)
            stft_vectors.append(stft_vector)

        stft_vectors = torch.stack([torch.from_numpy(vec) for vec in stft_vectors])  # Convert to tensor
        labels = torch.tensor(labels)

        return stft_vectors, labels



# Train function

In [ ]:
def train(model, dataset, batch_size, num_epochs, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    if not isinstance(dataset, Dataset):
        raise ValueError("The dataset parameter should be an instance of torch.utils.data.Dataset.")

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in data_loader:
            inputs,labels = batch[0],batch[1]
            inputs = inputs.unsqueeze(1)
            
            # Extract the inputs and targets
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        average_loss = running_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")

In [ ]:
folder_path="data/fma_small_stft/train/"
file_list = os.listdir(folder_path)
file_paths = [os.path.join(folder_path, file_name) for file_name in file_list]
print(file_paths)
dataset = MyDataset(file_paths, tr_labels)

In [ ]:
model = NNet1()
train(model, dataset, batch_size=16, num_epochs=10, learning_rate=0.001)

# Network Architecture Definition (nnet2)